Sources of dirty data:
- user entry errors
- poorly applied coding standards
- different schemas for the same type of item
- legacy system
- evolving applications
- no unique identifiers
- data migration
- programmer error
- corruption in transmission

Measures of data quality:
- validity: a degree entries conform to a schema 
- accuracy: conform to gold standard (correct entry?)
- completeness: all records?
- consistency: matches other data
- uniformity: same units

Blueprint (procedure)  for cleaning:
-  audit your data (programatically check using validation rules, prepare report, run statistical analysis i.e. to search for outliers
- create a data cleaning plan (identity causes, define set of operations, run tests)
- execute the plan (run the script)
- manually correct what was unable to be cleaned 
- .. and iterate the whole process! 

**re** Regular expression operations

What does it mean: ** street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE) **
- `r"\n"` is a two-character string containing '\' and 'n', while "\n" is a one-character string containing a newline.
- `\S\`  Matches any character which is not a whitespace character.
- `+` Causes the resulting RE to match 1 or more repetitions of the preceding RE. ab+ will match ‘a’ followed by any non-zero number of ‘b’s; it will not match just ‘a’.
- `backslash character "\"` indicates special forms or allow special characters to be used without invoking (**INVOKING - PRZYWOŁYWAĆ**) their special meaning 



- `.` (Dot.) In the default mode, this matches any character except a newline.
- `?` Causes the resulting RE to match 0 or 1 repetitions of the preceding RE. ab? will match either ‘a’ or ‘ab’.
- $ Matches the end of the string or just before the newline at the end of the string, and in MULTILINE mode also matches before a newline.

**COMPILE - TO PUT TOGETHER**

**HOLISTIC -  COMPREHENSION OF THE PARTS OF STH AS A WHOLE** The definition of holistic is relating to the idea that things should be studied as a whole and not just as a sum of their parts. An example of holistic is health care that focuses on the health of the entire body and mind and not just parts of the body.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
from collections import defaultdict
import re

osm_file = open("chicago.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name) # Scan through string looking for the first location where the regular expression pattern produces a match, and return a corresponding match object.
    if m:                                  # = if not None (there IS a match)
        street_type = m.group()            # A group() expression returns one or more subgroups of the match. 

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print("%s: %d" % (k, v))

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

if __name__ == '__main__':
    audit()

## **Auditing validity**
- mandatory fields
- unique value fields (i.e. user name)
- foreign-key constraints (i.e. product records must have a refence to a manufacturer, that has to exists in a the database)
- cross-field constraints (i.e. start date must come before the end date; a field is a result of an operation of other fields)
- data type or data structure
- regular expressions (some particular patterns)
- some range (numeric, or set membership i.e. S, M, L sizes)


In [18]:
"""
Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- check if the field "productionStartYear" contains a year
- check if the year is in range 1886-2014
- convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above, 
  write that line to the output_bad file
- discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.

You can write helper functions for checking the data and writing the files, but we will call only the 
'process_file' with 3 arguments (inputfile, output_good, output_bad).
"""
import csv
import pprint

INPUT_FILE = 'autos.csv'

# def is_a_year(input_row, key):
#     return (input_row[key] is int) and len(input_row[key]) == 4

def is_in_range(input_row, key):
    return str(input_row[key]) > '1886' and str(input_row[key]) < '2014'
    
def process_file(input_file, output_good, output_bad):

    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        
        #COMPLETE THIS FUNCTION
        good = []
        bad = []
        key = "productionStartYear"
        
        for row in reader:
            row[key] = row[key][:4]
            
            if row["URI"].startswith("http://dbpedia.org"):
                if is_in_range(row, key):
                    good.append(row)

                else:
                    bad.append(row)
    
    print("good", good)
    print("\nbad", bad)

    # This is just an example on how you can use csv.DictWriter
    # Remember that you have to output 2 files
    with open("output_good.csv", "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in good:
            writer.writerow(row)
            
    with open("output_bad.csv", "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in bad:
            writer.writerow(row)

def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)


if __name__ == "__main__":
    test()

good [OrderedDict([('URI', 'http://dbpedia.org/resource/Mazda_MX-5'), ('rdf-schema#label', 'Mazda MX-5'), ('rdf-schema#comment', 'The MX-5 also known as Miata in North America and Eunos Roadster in Japan is a lightweight two-seater roadster of front-engine rear-wheel-drive layout built by Mazda in Hiroshima Japan. The model was introduced in 1989 at the Chicago Auto Show.'), ('assembly_label', '{Hiroshima|Japan}'), ('assembly', '{http://dbpedia.org/resource/Hiroshima|http://dbpedia.org/resource/Japan}'), ('automobilePlatform_label', 'Mazda N platform'), ('automobilePlatform', 'http://dbpedia.org/resource/Mazda_N_platform'), ('bodyStyle_label', 'NULL'), ('bodyStyle', 'NULL'), ('class_label', 'Roadster (automobile)'), ('class', 'http://dbpedia.org/resource/Roadster_(automobile)'), ('designCompany_label', 'NULL'), ('designCompany', 'NULL'), ('designer_label', 'NULL'), ('designer', 'NULL'), ('engine_label', 'NULL'), ('engine', 'NULL'), ('fuelCapacity', 'NULL'), ('height', 'NULL'), ('layout

# Auditing completeness
- you don't know what you don't know
- missing records
- similar solution to accuracy
- need reference data

## Auditing consistency 
(two different entries cobntradict one another)
- which data source is more reliable?
- which collection method is more reliable (i.e. location: GPS, user enter data, IP-based reading?)
- data collected in a different point in time may be the same for two different entities

**FILE FOR BANKRUPTCY, RECOVER FROM BANKRUPTCY**

## Auditing uniformity
(same units of measurement)
- 